In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/validation-original-aihub/VS_0975.zip
/kaggle/input/validation-original-aihub/VS_1001.zip
/kaggle/input/validation-original-aihub/VS_0929.zip
/kaggle/input/validation-original-aihub/VS_0913.zip
/kaggle/input/validation-original-aihub/VS_0954.zip
/kaggle/input/validation-original-aihub/VS_0995.zip
/kaggle/input/validation-original-aihub/VS_0932.zip
/kaggle/input/validation-original-aihub/VS_0910.zip
/kaggle/input/validation-original-aihub/VS_0981.zip
/kaggle/input/validation-original-aihub/VS_0907.zip
/kaggle/input/validation-original-aihub/VS_0921.zip
/kaggle/input/validation-original-aihub/VS_0920.zip
/kaggle/input/validation-original-aihub/VS_0941.zip
/kaggle/input/validation-original-aihub/VS_0994.zip
/kaggle/input/validation-original-aihub/VS_0926.zip
/kaggle/input/validation-original-aihub/VS_0895.zip
/kaggle/input/validation-original-aihub/VS_0968.zip
/kaggle/input/validation-original-aihub/VS_0965.zip
/kaggle/input/validation-original-aihub/VS_0962.zip
/kaggle/inpu

> # Upload this ipynb to GitHub under your GitHub account won-hj. This can only be undone directly on GitHub.

MIT License
Copyright (c) <year> <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

참고?
https://www.kaggle.com/code/diaaessam/face-verification-and-recognition 
https://www.kaggle.com/code/atulanandjha/face-recognition-starter-trick-code-with-output
https://www.kaggle.com/datasets/atulanandjha/lfwpeople


#import 
# 이미지 로드 및 전처리하기
- 먼저 고급 Keras 사전 처리 유틸리티(예: tf.keras.utils.image_dataset_from_directory) 및 레이어(예: tf.keras.layers.Rescaling)를 사용하여 디스크에서 이미지 디렉터리를 읽습니다.
- 다음으로 tf.data를 사용하여 처음부터 고유한 입력 파이프라인을 작성합니다.
- 마지막으로 TensorFlow Datasets에서 사용할 수 있는 대규모 카탈로그에서 데이터세트를 다운로드합니다.

In [36]:
import keras
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [37]:
import zipfile
import os
import glob

filename = {'train-label':'TL_',
            'train-original':'TS_', 
            'validation-labelling':'VL_', 
            'validation-original':'VS_'}

input_path = '/kaggle/input/'
working_path = '/kaggle/working/'  #문제생기면 여기서 발생 

for k,v in filename.items():
    file_path = input_path + {}-aihub/'.format(k)   
    #.join(glob.glob('.zip'))#.join('-aihub/TL_').join(glob.glob('.zip'))
    print(file_path)
    dir_path = working_path + {}/'.format(k)
    print(dir_path)
    
    
    # 지정된 디렉토리가 없으면 생성합니다.
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    #print(os.getcwd())
    # zip 파일 압축 해제
    # 각 압축파일마다 압축해제 
    files = os.listdir(file_path)
    print(files)
    
    for i in range(len(files)):
        with zipfile.ZipFile(file_path+files[i], 'r') as zip_ref:
            zip_ref.extractall(dir_path)

SyntaxError: unterminated string literal (detected at line 14) (3057066489.py, line 14)

In [1]:
#print(os.dirlist('kaggle/working/train-original'))
#!pwd
#!rm *.json
#!ls 
#!ls train-labelling
#!rm -rf train-lebelling
#!ls validation-labelling
#import os
#os.removedirs('/kaggle/working/train-labelling')
#!ls /kaggle/working/

state.db     train-original	   validation-original
train-label  validation-labelling


In [18]:
#데이터셋의 데이터 구조를 출력 - label 
!cat /kaggle/working/train-label/0016_1968_55_00000060_D.json

'''
json 전처리 코드
https://velog.io/@parksh089g/json%ED%98%95%EC%8B%9D-Dataset-%EC%A0%84%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0
0016_1968_55_00000060_D.json -> id_출생년도_나이_
** 크기 조정 후 바운딩 박스나 랜드마크가 변경되는지 확인 **
{
    "filename": "0016_1968_55_00000060_D", #id_birth_age-past
    "id": 16,
    "birth": 1968,
    "age_now": 55,
    "age_past": 55,
    "format": "png",
    "imgsize": "1.324MB",
    "width": 1080,
    "height": 1440,
    "device": "digital",
    "gender": "male",
    "annotation": [
        {
            "box": {
                "x": 3.047921471137073,
                "y": 469.34504908053935,
                "w": 474.59417858244757,
                "h": 519.9905464947254
            },
            "landmark": [
                [
                    311.88763427734375,
                    722.7200927734375
                ],
                [
                    169.94879515695465,
                    559.9612768464077
                ],
                [
                    397.5303955078125,
                    587.8244458053117
                ],
                [
                    165.16746397340108,
                    823.2084790084367
                ],
                [
                    355.03024047511775,
                    823.2270239167203
                ]
            ]
        }
    ]
}

'''

{
    "filename": "0016_1968_55_00000060_D",
    "id": 16,
    "birth": 1968,
    "age_now": 55,
    "age_past": 55,
    "format": "png",
    "imgsize": "1.324MB",
    "width": 1080,
    "height": 1440,
    "device": "digital",
    "gender": "male",
    "annotation": [
        {
            "box": {
                "x": 3.047921471137073,
                "y": 469.34504908053935,
                "w": 474.59417858244757,
                "h": 519.9905464947254
            },
            "landmark": [
                [
                    311.88763427734375,
                    722.7200927734375
                ],
                [
                    169.94879515695465,
                    559.9612768464077
                ],
                [
                    397.5303955078125,
                    587.8244458053117
                ],
                [
                    165.16746397340108,
                    823.2084790084367
                ],
                [
                    

In [35]:
#데이터셋의 데이터 구조를 출력 - label 
!cat /kaggle/working/train-original/0009_1991_14_00000050_F.png
#이미지가 깨져서 나온다 

cat: /kaggle/working/train-original/0009_1991_14_00000050_F.png: No such file or directory


In [9]:
#디렉토리 개수 확인
import os


filename = {'train-label':'TL_',
            'train-original':'TS_', 
            'validation-labelling':'VL_', 
            'validation-original':'VS_'}
dirname = ['train-label', 'train-original', 'validation-labelling', 'validation-original']
len_files = []

ts_path = '/kaggle/working/train-original'#working_path + dir_list[0]
for key in dirname:
    p = '/kaggle/working/'+str(key)
    len_files.append(len(os.listdir(p))) 

print(len_files)
#!tree /kaggle/working/

[0, 2763, 5000, 4000]
